In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import pandas as pd

In [6]:
# Mount Google Drive
import os
from google.colab import drive
drive.mount('drive')
os.chdir("/content/drive/My Drive/19-20/Spring/Data88/Supreme_Court")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
#change: file id, name of csv file 

#find the file id of the corpus.csv file in your drive. This is YOUR file, not mine 
#this can be found by creating a shareable link for the file in your drive. The id is the last part of the link that comes after id = ... 
#downloaded = drive.CreateFile({'id':'1-4ZVz38zoKFFVvQchcfX5sgjrjcGIu_L'}) 
#downloaded.GetContentFile('2_col_samp.json')
file_dictionary = pd.read_json('2_col_warren.json').dropna()

Because we are limited by the hardware for visualization, I have used a sample of the cases of size 500.

In [0]:
file = file_dictionary.values   ## this cell is for putting all the files in a dictionary shape.
file_dictionary = {}
for i in file:
  file_dictionary[i[0]] = i[1]

In [11]:
#change: text file name to one of your txt files in corpus.csv

#check with one of your text files to see if it works 
file_dictionary["MASLOWSKY v. CASSIDY, 385 U.S. 11 (1966)"] ## Now we can check the key to be the file name, the content is matched to the key

'\n\nUnited States Supreme Court\nCAPELOUTO v. ORKIN EXTERMINATING CO.(1966)\nNo. 316\nArgued: Decided: October 10, 1966\n\n\n\n\n 183 So.2d 532, appeal dismissed.\n\nWilfred C. Varn for appellant.\n\nJ. Lewis Hall for appellee.\n\n\n\nPER CURIAM.\n\n\nThe motion to dismiss is granted and the appeal is dismissed for want of a substantial federal question.\n\n\n\n\n                  MASLOWSKY v. CASSIDY, \n<flCite id="/us-supreme-court/385/11#">385 \nU.S. 11\n</flCite> (1966)\n\n\n385 \nU.S. 11\n (1966)\n\n">\n\n\nU.S. Supreme Court\n\n\n                  MASLOWSKY v. CASSIDY, \n385 \nU.S. 11\n (1966)\n\n\n\n385 \nU.S. 11\n\n          MASLOWSKY ET AL. v. CASSIDY, CHAIRMAN, ILLINOIS HOUSE OF\n\n                     REPRESENTATIVES COMMISSION, ET AL.\n\n   APPEAL FROM THE SUPREME COURT OF ILLINOIS.      No. 574.\nDecided October 10, 1966.\n\n\n\n\n\n 34 Ill. 2d 456, 216 N. E. 2d 669, appeal dismissed.\n\nAlbert E. Jenner, Jr., and Thomas P. Sullivan for appellants.\n\nOwen Rall for appell

get help from: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [13]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
#change: add your languages! 

### Get the stopwords for the languages of your corpus from nltk package
english_stopwords = stopwords.words('english')
corpus_stopwords = ["court", "see", "statute", "pp"] #add stopwords unique to your corpus 


In [0]:
## This is a function designed to remove all the stopwords in our text
def preprocess(text): # one little problem, gensim.utils.simple_preprocess gets rid off "I" automatically. 
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
          if token not in english_stopwords:
            if token not in corpus_stopwords:
              #continue nesting stopwords for different languagses here 
                result.append(token)
    return result

In [0]:
for key in file_dictionary:
  #print("working on file "+ key)
  file_dictionary[key] = preprocess(file_dictionary[key])

In [0]:
#change: the name of the text file to one of your files 
#test_file = file_dictionary['WISCONSIN CENTRAL LTD. ET AL. v. UNITED STATES'] # an example to show what the file is stored in the dictionrary after preprocessed.

In [0]:
processed_files = []
for key in file_dictionary:
  processed_files.append(file_dictionary[key])

In [0]:
dictionary = gensim.corpora.Dictionary(processed_files)

In [0]:
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)

In [0]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_files]

In [0]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [0]:
#change: set topic number and word number to whatever you want 

topic_number = 15 # how many topic you want
word_number = 20

In [0]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=topic_number, id2word=dictionary, passes=2, workers=2)

In [27]:
for idx, topic in lda_model.show_topics(num_topics = topic_number, num_words= 5, log=False, formatted=True): ## change num_words to see number of words you want to see in a topic
    print('Topic: {} \nWords: {}'.format(idx+1, topic))

Topic: 1 
Words: 0.005*"carriers" + 0.005*"carrier" + 0.004*"rates" + 0.004*"contempt" + 0.004*"competition"
Topic: 2 
Words: 0.006*"milk" + 0.004*"sales" + 0.003*"bargaining" + 0.003*"antitrust" + 0.003*"contempt"
Topic: 3 
Words: 0.006*"lands" + 0.006*"income" + 0.004*"wife" + 0.003*"estate" + 0.003*"corpus"
Topic: 4 
Words: 0.006*"patent" + 0.006*"religious" + 0.004*"juvenile" + 0.003*"election" + 0.003*"religion"
Topic: 5 
Words: 0.006*"military" + 0.004*"merger" + 0.004*"citizenship" + 0.003*"inclusion" + 0.003*"communist"
Topic: 6 
Words: 0.006*"strike" + 0.006*"bargaining" + 0.004*"membership" + 0.004*"grand" + 0.004*"carrier"
Topic: 7 
Words: 0.007*"confession" + 0.004*"income" + 0.003*"school" + 0.003*"insurance" + 0.003*"habeas"
Topic: 8 
Words: 0.006*"communist" + 0.003*"conspiracy" + 0.003*"death" + 0.003*"collective" + 0.003*"bargaining"
Topic: 9 
Words: 0.007*"competition" + 0.006*"market" + 0.006*"merger" + 0.004*"antitrust" + 0.004*"stock"
Topic: 10 
Words: 0.006*"natur

In [28]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx+1, topic))

Topic: 1 
Words: 0.005*"carriers" + 0.005*"carrier" + 0.004*"rates" + 0.004*"contempt" + 0.004*"competition" + 0.004*"traffic" + 0.004*"market" + 0.004*"discrimination" + 0.004*"transportation" + 0.003*"intrastate"
Topic: 2 
Words: 0.006*"milk" + 0.004*"sales" + 0.003*"bargaining" + 0.003*"antitrust" + 0.003*"contempt" + 0.003*"transcript" + 0.003*"conspiracy" + 0.003*"suits" + 0.002*"prices" + 0.002*"collective"
Topic: 3 
Words: 0.006*"lands" + 0.006*"income" + 0.004*"wife" + 0.003*"estate" + 0.003*"corpus" + 0.003*"waters" + 0.003*"arbitration" + 0.003*"death" + 0.003*"water" + 0.003*"indian"
Topic: 4 
Words: 0.006*"patent" + 0.006*"religious" + 0.004*"juvenile" + 0.003*"election" + 0.003*"religion" + 0.003*"ship" + 0.003*"vessel" + 0.003*"sunday" + 0.003*"negligence" + 0.003*"owner"
Topic: 5 
Words: 0.006*"military" + 0.004*"merger" + 0.004*"citizenship" + 0.003*"inclusion" + 0.003*"communist" + 0.003*"forces" + 0.003*"martial" + 0.003*"penn" + 0.002*"land" + 0.002*"armed"
Topic: 6 

In [0]:
top = {}
for i in range(topic_number): 
  top.update({i+1:[]})

In [0]:
for key in file_dictionary:
  topics = sorted(lda_model[bow_corpus[processed_files.index(file_dictionary[key])]], key = lambda tup: -1*tup[1])
  for i in topics:
    top_number = i[0]+1
    inside = [key, i[1]]
    if i[1] > 0.4: ## change the threshold for topic weights
      top[top_number].append(inside)

Then we make a dictionary (top) for the topics (topics) have been generated for each document (key) in the document set (fiile_dictionary)

In [0]:
Source = []
Target = []
Source_weight = []
Target_weight = []
Topic = []

In [59]:
len(top)

15

In [0]:
for i in range(1, len(top)+1):
  for j in range(len(top[i])):
    for k in range(j+1, len(top[i])):
      if top[i][j][1] >= top[i][k][1]:
        Source.append(top[i][j][0])
        Target.append(top[i][k][0])
        Source_weight.append(top[i][j][1])
        Target_weight.append(top[i][k][1])
      else:
        Source.append(top[i][k][0])
        Target.append(top[i][j][0])
        Source_weight.append(top[i][k][1])
        Target_weight.append(top[i][j][1])
      Topic.append(i)

In [0]:
result = pd.DataFrame({'Source':Source, 'Target':Target, 'Source_Weight':Source_weight, 'Target_Weight': Target_weight, 'Topic': Topic})

In [0]:
Type = ['directed'] * len(result)

In [0]:
result = pd.DataFrame({'Source':Source, 'Target':Target, 'Source_Weight':Source_weight, 'Target_Weight': Target_weight, 'Type': Type, 'Topic': Topic})

In [64]:
Topic[-1]

15

In [65]:
result 

,Source,Target,Source_Weight,Target_Weight,Type,Topic
0,"CHICAGO v. WILLETT CO., 344 U.S. 574 (1953)","GORDON v. UNITED STATES, 344 U.S. 414 (1953)",0.782773,0.602274,directed,1
1,"BODE v. BARRETT, 344 U.S. 583 (1953)","GORDON v. UNITED STATES, 344 U.S. 414 (1953)",0.740059,0.602274,directed,1
2,"GORDON v. UNITED STATES, 344 U.S. 414 (1953)",ISSERMAN v. ETHICS COMMITTEE OF ESSEX COUNTY B...,0.602274,0.583417,directed,1
3,"IN RE ISSERMAN, 345 U.S. 286 (1953)","GORDON v. UNITED STATES, 344 U.S. 414 (1953)",0.607340,0.602274,directed,1
4,"AUTOMATIC CANTEEN CO. v. F. T. C., 346 U.S. 61...","GORDON v. UNITED STATES, 344 U.S. 414 (1953)",0.999339,0.602274,directed,1
...,...,...,...,...,...,...
434977,"OESTEREICH v. SELECTIVE SERVICE BD., 393 U.S. ...","JOHNSON v. POWELL, 393 U.S. 920 (1968)",0.562627,0.545851,directed,15
434978,"CLARK v. GABRIEL, 393 U.S. 256 (1968)","JOHNSON v. POWELL, 393 U.S. 920 (1968)",0.979022,0.545851,directed,15
434979,"STAMLER v. WILLIS, 393 U.S. 217 (1968)","OESTEREICH v. SELECTIVE SERVICE BD., 393 U.S. ...",0.955555,0.562627,directed,15
434980,"CLARK v. GABRIEL, 393 U.S. 256 (1968)","STAMLER v. WILLIS, 393 U.S. 217 (1968)",0.979022,0.955555,directed,15


How many rows can a Google Sheet have?
Originally Answered: How many rows can you have in a google spreadsheet before performance becomes a serious problem? Short answer: 400,000 cells - http://docs.google.com/support/b... - Note the distinction between rows and cells. This means that you could have 40,000 rows of 10 columns or 80,000 rows of 5 columns.Oct 12, 2010

In [0]:
#removed typo in file name, needs to be My\ Drive not My Drive (backslash neccessary)

result.to_csv('topic_edge_list_warren.csv', index=False)

In [55]:
!ls

 2_col_earl_samp.json	    all_cases.tar.gz	    SupremeCourt.ipynb
'2_col_warren (1).gsheet'   big_table.json	    topic_edge_list_warren.csv
'2_col_warren (2).gsheet'   big_table_warren.json   Topic_Model
 2_col_warren.csv	    cache		    warren_cases
 2_col_warren.gsheet	    Doc2Vec		    Word2Vec
 2_col_warren.json	    docs
